**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# Istio Canary

## Hello Istio

Very basic traffic management
https://istio.io/blog/2018/incremental-traffic-management/



In [ ]:
kubectl label namespace default istio-injection=enabled

## Create a test pod

In [ ]:
kubectl apply -f - <<EOF
apiVersion: v1
kind: Service
metadata:
  name: sleep
  labels:
    app: sleep
spec:
  ports:
  - port: 80
    name: http
  selector:
    app: sleep
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: sleep
spec:
  replicas: 1
  selector:
    matchLabels:
      app: sleep
  template:
    metadata:
      labels:
        app: sleep
    spec:
      containers:
      - name: sleep
        image: pstauffer/curl
        command: ["/bin/sleep", "3650d"]
        imagePullPolicy: IfNotPresent
---
EOF

In [ ]:
export SLEEP_POD=$(kubectl get pod -l app=sleep \
  -o jsonpath={.items..metadata.name})
echo $SLEEP_POD

In [ ]:
kubectl get crd

In [ ]:
kubectl get destinationrule 

In [ ]:
kubectl get virtualservice

In [ ]:
kubectl get deployments

In [ ]:
kubectl get pods

In [ ]:
kubectl get pod $SLEEP_POD -o yaml | grep --color=always  -E '$|.*image:.*'

# Admission Controlers and Sidecars

- Istio works as an [ Admission Controller](https://kubernetes.io/blog/2019/03/21/a-guide-to-kubernetes-admission-controllers/)
- adding a [Sidecar Pattern](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/45406.pdf)
- [Kubernetes Patterns](https://www.redhat.com/cms/managed-files/cm-oreilly-kubernetes-patterns-ebook-f19824-201910-en.pdf)
- [Envoy Proxy](https://www.envoyproxy.io/)
- [Patterns with Envoy Proxy](https://blog.christianposta.com/microservices/00-microservices-patterns-with-envoy-proxy-series/)

In [11]:
kubectl exec $SLEEP_POD -- curl  heise.de 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>301 Moved Permanently</title>
</head><body>
<h1>Moved Permanently</h1>
<p>The document has moved <a href="https://www.heise.de/">here</a>.</p>
</body></html>
100   229  100   229    0     0   3693      0 --:--:-- --:--:-- --:--:--  3693


# Istio Sample Application

In [12]:
ls 

 AccountWithRBAC.ipynb       OpenPolicyAgent.ipynb
 AuditLogs.ipynb             OpenShift
 Calico-Iptables.ipynb       Overview.ipynb
 ClusterOverview.ipynb       PodSecurityPolicyBasic.ipynb
 ContainerdCloudbomb.ipynb   PodSecurityPolicyPermissive.ipynb
 CrioBomb.ipynb              PodSecurityPolicyStrict.ipynb
 Harbor.ipynb               'Postgres Zalando.ipynb'
 Hardening-NGinx.ipynb       README.md
 Helm.ipynb                  ServiceAccountToken.ipynb
 Ingress.ipynb               Unprivileged.ipynb
 IngressKong.ipynb           default-rbac.png
 IngressKongStart.ipynb      defaut-rbac.dot
 IngressStart.ipynb          docker-hypervisor.png
 Istio-Traffic-Mgmt.ipynb    istio-1.12.2
 Istio1.8.2.ipynb            kube-bench
 IstioHack.ipynb             kube-bench.ipynb
 Keycloak.ipynb              kubernetes-client.png
 Kube-Scan-Cloudbomb.ipynb   minikube.dot
 KubeLogin.ipynb             minikube.png
 KubernetesBasics.ipynb      my-namespace-rbac.dot
 License.ipynb               my-nam

In [13]:
cd istio-1.12.2

In [14]:
pwd

/minikube-host/notebooks/training-kubernetes-security/istio-1.12.2


In [15]:
kubectl apply -f samples/bookinfo/platform/kube/bookinfo.yaml

service/details created
serviceaccount/bookinfo-details created
deployment.apps/details-v1 created
service/ratings created
serviceaccount/bookinfo-ratings created
deployment.apps/ratings-v1 created
service/reviews created
serviceaccount/bookinfo-reviews created
deployment.apps/reviews-v1 created
deployment.apps/reviews-v2 created
deployment.apps/reviews-v3 created
service/productpage created
serviceaccount/bookinfo-productpage created
deployment.apps/productpage-v1 created


In [22]:
kubectl get pods 

NAME                              READY   STATUS    RESTARTS   AGE
details-v1-79f774bdb9-8ntdr       2/2     Running   0          64s
productpage-v1-6b746f74dc-7h84n   2/2     Running   0          64s
ratings-v1-b6994bb9-brnkv         2/2     Running   0          64s
reviews-v1-545db77b95-lzpkt       2/2     Running   0          64s
reviews-v2-7bf8c9648f-g7859       2/2     Running   0          64s
reviews-v3-84779c7bbc-qh6gb       2/2     Running   0          64s
sleep-56bc8d768-trznv             2/2     Running   0          10m


In [23]:
kubectl get services

NAME          TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)    AGE
details       ClusterIP   10.103.164.205   <none>        9080/TCP   3m36s
kubernetes    ClusterIP   10.96.0.1        <none>        443/TCP    41m
productpage   ClusterIP   10.106.238.118   <none>        9080/TCP   3m36s
ratings       ClusterIP   10.104.234.149   <none>        9080/TCP   3m36s
reviews       ClusterIP   10.102.199.58    <none>        9080/TCP   3m36s
sleep         ClusterIP   10.110.217.22    <none>        80/TCP     12m


In [24]:
kubectl exec "$(kubectl get pod -l app=ratings -o jsonpath='{.items[0].metadata.name}')" \
  -c ratings -- curl -s productpage:9080/productpage | grep --color=always -E "$|<title>.*</title>"

<!DOCTYPE html>
<html>
  <head>
    <title>Simple Bookstore App</title>
<meta charset="utf-8">
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<meta name="viewport" content="width=device-width, initial-scale=1.0">

<!-- Latest compiled and minified CSS -->
<link rel="stylesheet" href="static/bootstrap/css/bootstrap.min.css">

<!-- Optional theme -->
<link rel="stylesheet" href="static/bootstrap/css/bootstrap-theme.min.css">

  </head>
  <body>
    
    

<nav class="navbar navbar-inverse navbar-static-top">
  <div class="container">
    <div class="navbar-header">
      <a class="navbar-brand" href="#">BookInfo Sample</a>
    </div>
    
    <button type="button" class="btn btn-default navbar-btn navbar-right" data-toggle="modal" href="#login-modal">Sign
      in</button>
    
  </div>
</nav>

<!---
<div class="navbar navbar-inverse navbar-fixed-top">
  <div class="container">
    <div class="navbar-header pull-left">
      <a class="navbar-brand" href="#">Microservices Fabric Bo

In [25]:
cat samples/bookinfo/networking/bookinfo-gateway.yaml

apiVersion: networking.istio.io/v1alpha3
kind: Gateway
metadata:
  name: bookinfo-gateway
spec:
  selector:
    istio: ingressgateway # use istio default controller
  servers:
  - port:
      number: 80
      name: http
      protocol: HTTP
    hosts:
    - "*"
---
apiVersion: networking.istio.io/v1alpha3
kind: VirtualService
metadata:
  name: bookinfo
spec:
  hosts:
  - "*"
  gateways:
  - bookinfo-gateway
  http:
  - match:
    - uri:
        exact: /productpage
    - uri:
        prefix: /static
    - uri:
        exact: /login
    - uri:
        exact: /logout
    - uri:
        prefix: /api/v1/products
    route:
    - destination:
        host: productpage
        port:
          number: 9080


In [26]:
kubectl apply -f samples/bookinfo/networking/bookinfo-gateway.yaml

gateway.networking.istio.io/bookinfo-gateway created
virtualservice.networking.istio.io/bookinfo created


In [27]:
export PATH=$PATH:$(pwd)/bin

In [28]:
istioctl analyze


✔ No validation issues found when analyzing namespace: default.


In [29]:
export INGRESS_PORT=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.spec.ports[?(@.name=="http2")].nodePort}')
export SECURE_INGRESS_PORT=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.spec.ports[?(@.name=="https")].nodePort}')
export INGRESS_HOST=$(hostname -i)
echo INGRESS_PORT=$INGRESS_PORT
echo SECURE_INGRESS_PORT=$SECURE_INGRESS_PORT
echo INGRESS_HOST=$INGRESS_HOST

INGRESS_PORT=30844
SECURE_INGRESS_PORT=31783
INGRESS_HOST=192.168.49.2


In [30]:
export GATEWAY_URL=$INGRESS_HOST:$INGRESS_PORT
echo "http://$GATEWAY_URL/productpage"

http://192.168.49.2:30844/productpage


In [31]:
kubectl apply -f samples/addons

serviceaccount/grafana created
configmap/grafana created
service/grafana created
deployment.apps/grafana created
configmap/istio-grafana-dashboards created
configmap/istio-services-grafana-dashboards created
deployment.apps/jaeger created
service/tracing created
service/zipkin created
service/jaeger-collector created
serviceaccount/kiali created
configmap/kiali created
clusterrole.rbac.authorization.k8s.io/kiali-viewer created
clusterrole.rbac.authorization.k8s.io/kiali created
clusterrolebinding.rbac.authorization.k8s.io/kiali created
role.rbac.authorization.k8s.io/kiali-controlplane created
rolebinding.rbac.authorization.k8s.io/kiali-controlplane created
service/kiali created
deployment.apps/kiali created
serviceaccount/prometheus created
configmap/prometheus created
clusterrole.rbac.authorization.k8s.io/prometheus created
clusterrolebinding.rbac.authorization.k8s.io/prometheus created
service/prometheus created
deployment.apps/prometheus created


In [36]:
kubectl get pods -n istio-system

NAME                                    READY   STATUS    RESTARTS   AGE
grafana-6ccd56f4b6-8s9qf                1/1     Running   0          100s
istio-egressgateway-c9cbbd99f-rzckb     1/1     Running   0          39m
istio-ingressgateway-7c8bc47b49-mq4d8   1/1     Running   0          39m
istiod-765596f7ff-m9f8l                 1/1     Running   0          39m
jaeger-5d44bc5c5d-f2jss                 1/1     Running   0          100s
kiali-79b86ff5bc-cszx6                  1/1     Running   0          99s
prometheus-64fd8ccd65-4wgmz             2/2     Running   0          99s


In [37]:
kubectl get all -n istio-system

NAME                                        READY   STATUS    RESTARTS   AGE
pod/grafana-6ccd56f4b6-8s9qf                1/1     Running   0          3m14s
pod/istio-egressgateway-c9cbbd99f-rzckb     1/1     Running   0          40m
pod/istio-ingressgateway-7c8bc47b49-mq4d8   1/1     Running   0          40m
pod/istiod-765596f7ff-m9f8l                 1/1     Running   0          41m
pod/jaeger-5d44bc5c5d-f2jss                 1/1     Running   0          3m14s
pod/kiali-79b86ff5bc-cszx6                  1/1     Running   0          3m13s
pod/prometheus-64fd8ccd65-4wgmz             2/2     Running   0          3m13s

NAME                           TYPE           CLUSTER-IP       EXTERNAL-IP     PORT(S)                                                                      AGE
service/grafana                ClusterIP      10.110.160.59    <none>          3000/TCP                                                                     3m14s
service/istio-egressgateway    ClusterIP      10.102.

In [38]:
istioctl dashboard kiali --address $(hostname -i) | sed s/localhost/$(hostname -i)/

http://192.168.49.2:20001/kiali
Failed to open browser; open http://192.168.49.2:20001/kiali in your browser.



## Cleanup

In [39]:
kubectl delete -f samples/addons -f samples/bookinfo/platform/kube/bookinfo.yaml

serviceaccount "grafana" deleted
configmap "grafana" deleted
service "grafana" deleted
deployment.apps "grafana" deleted
configmap "istio-grafana-dashboards" deleted
configmap "istio-services-grafana-dashboards" deleted
deployment.apps "jaeger" deleted
service "tracing" deleted
service "zipkin" deleted
service "jaeger-collector" deleted
serviceaccount "kiali" deleted
configmap "kiali" deleted
clusterrole.rbac.authorization.k8s.io "kiali-viewer" deleted
clusterrole.rbac.authorization.k8s.io "kiali" deleted
clusterrolebinding.rbac.authorization.k8s.io "kiali" deleted
role.rbac.authorization.k8s.io "kiali-controlplane" deleted
rolebinding.rbac.authorization.k8s.io "kiali-controlplane" deleted
service "kiali" deleted
deployment.apps "kiali" deleted
serviceaccount "prometheus" deleted
configmap "prometheus" deleted
clusterrole.rbac.authorization.k8s.io "prometheus" deleted
clusterrolebinding.rbac.authorization.k8s.io "prometheus" deleted
service "prometheus" deleted
deployment.apps "prometh

In [40]:
kubectl get all

NAME                            READY   STATUS        RESTARTS   AGE
pod/ratings-v1-b6994bb9-brnkv   2/2     Terminating   0          50m
pod/sleep-56bc8d768-trznv       2/2     Running       0          59m

NAME                 TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.96.0.1       <none>        443/TCP   87m
service/sleep        ClusterIP   10.110.217.22   <none>        80/TCP    59m

NAME                    READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/sleep   1/1     1            1           59m

NAME                              DESIRED   CURRENT   READY   AGE
replicaset.apps/sleep-56bc8d768   1         1         1       59m
